In [1]:
%pylab
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
import scipy
import pandas as pd
import seaborn as sns
path =  (os.path.abspath(os.path.join(r'.', os.pardir)))
import sys
sys.path.append(path)
from nuclei_segmentation import otsu as ot
from nuclei_segmentation import dicescore as dsc
from PIL import Image
import os
import os.path

Using matplotlib backend: <object object at 0x00000205C391E6C0>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Set ground truth images from N2DH-GOWT1 dataset and N2DL-HeLa dataset to 0 and 1

def binarize(x):
    img = x.copy()

    for o in np.ndindex(img.shape):
        if img[o] > 0: 
           img[o] = 1
        else:
            img[o] = 0
    
    return img

## NIH3T3 Dataset

In [3]:
# Load raw images of NIH3T3-dataset

img_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/img")
for f in os.listdir(path):
    img_NIH3T3.append(imread(os.path.join(path , f)))

# Load ground truth images of NIH3T3-dataset
 
gt_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/gt")
for f in os.listdir(path):
    gt_NIH3T3.append(imread(os.path.join(path , f)))

In [4]:
# Make a copy of all images of each dataset / use as "no preprocessed images"
copy_img_NIH3T3 = img_NIH3T3.copy()

In [5]:
print(gt_NIH3T3)
shape(gt_NIH3T3)

[array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0

(18, 1024, 1344)

In [6]:
print(img_NIH3T3)

[array([[0.14509805, 0.15294118, 0.14901961, ..., 0.07843138, 0.07843138,
        0.07450981],
       [0.16078432, 0.14901961, 0.14901961, ..., 0.07843138, 0.08235294,
        0.08235294],
       [0.14901961, 0.14901961, 0.15686275, ..., 0.08235294, 0.08235294,
        0.07843138],
       ...,
       [0.08627451, 0.08235294, 0.08627451, ..., 0.09411765, 0.09019608,
        0.09019608],
       [0.08627451, 0.09019608, 0.08627451, ..., 0.09411765, 0.09019608,
        0.09411765],
       [0.08627451, 0.08235294, 0.08627451, ..., 0.09411765, 0.09411765,
        0.09019608]], dtype=float32), array([[0.08235294, 0.08235294, 0.08235294, ..., 0.12941177, 0.1254902 ,
        0.13333334],
       [0.07843138, 0.08235294, 0.08627451, ..., 0.12941177, 0.13333334,
        0.1254902 ],
       [0.08235294, 0.08627451, 0.08235294, ..., 0.12941177, 0.12941177,
        0.13333334],
       ...,
       [0.13333334, 0.13333334, 0.12156863, ..., 0.08627451, 0.08235294,
        0.08235294],
       [0.14901961

#### Median filter

In [15]:
# Apply median filter to all images of each dataset
# mit kleinerern Filtern läuft es schneller durch
# filter größen die man möchte
filter = [12, 13, 14, 15, 16, 17]
median_img_NIH3T3 = []
for j in range (0, len(filter)):
    
    for i in range(0, len(copy_img_NIH3T3)):
        median_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[j]))
    print("filtersize done")    

filtersize done
filtersize done
filtersize done
filtersize done
filtersize done
filtersize done


#### gaussian filter

In [14]:
#shape(copy_img_NIH3T3)
shape(median_img_NIH3T3)

(108, 1024, 1344)

In [24]:
# Apply gaussian filter to all images of each dataset
#filter größen die mach möchte 
filter_g = [7, 8 ,9 , 10, 11, 12]
gauss_img_NIH3T3 = []

for j in range (0, len(filter_g)):
    for i in range(0,len(copy_img_NIH3T3)):
        gauss_img_NIH3T3.append(scipy.ndimage.gaussian_filter(copy_img_NIH3T3[i] , filter_g[j]))
    print("filtersize done")
        


filtersize done
filtersize done
filtersize done
filtersize done
filtersize done
filtersize done


In [16]:
shape(gauss_img_NIH3T3)
shape(median_img_NIH3T3)

(108, 1024, 1344)

## Otsu thresholding

In [16]:
# Apply otsu thresholding to all images of each dataset - median filter
otsu_median_img_NIH3T3 = []
for i in range(0,len(median_img_NIH3T3)):
    otsu_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [i] , 256))
print(shape(otsu_median_img_NIH3T3))
print(otsu_median_img_NIH3T3)




(108, 1024, 1344)
[array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [

In [25]:
# Apply otsu thresholding to all images of each dataset - gaussian filter
otsu_gauss_img_NIH3T3 = []
for i in range(0,len(gauss_img_NIH3T3)):
    otsu_gauss_img_NIH3T3.append(ot.otsu_thresholding(gauss_img_NIH3T3[i] , 256))


In [9]:
gt6_NIH3T3 = []

path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/gt")
y=0
while y<6:
    for f in os.listdir(path):
        gt6_NIH3T3.append(imread(os.path.join(path , f)))
    y+=1

print(shape(gt6_NIH3T3))

(108, 1024, 1344)


## Dice score

In [17]:
# Dice score - NIH3T3-dataset
## median filter
dice_otsu_median_img_NIH3T3 = []

for i in range(0, len(otsu_median_img_NIH3T3)):
  dice_otsu_median_img_NIH3T3.append(dsc.dice(otsu_median_img_NIH3T3[i] , gt6_NIH3T3[i]))

print(dice_otsu_median_img_NIH3T3)


[0.9106564041529611, 0.8820733998112202, 0.8268482209444993, 0.7318057047800085, 0.7505206754410485, 0.666873922636296, 0.6734183350281524, 0.7205489066640851, 0.03145435272215811, 0.45837330774970686, 0.0, 0.6790666632440896, 3.517163759144626e-05, 0.5695515461935672, 0.6127163218988562, 0.054395327483482574, 0.06709385921330327, 0.7894065145997632, 0.9094515161961781, 0.8807962825174432, 0.826071107985156, 0.7299106553176201, 0.7483890044254329, 0.6885203152608366, 0.6033000876466696, 0.6908119509684427, 0.033777575540134934, 0.456808478084521, 0.0, 0.6776130602902554, 7.912644405760406e-05, 0.5677707714884977, 0.610995912247698, 0.05314685314685315, 0.06693522342902793, 0.7895243995731666, 0.9094190090903933, 0.880596107055961, 0.8255249885226481, 0.7298094897468379, 0.7489302967563837, 0.6879530075314786, 0.6027956635151179, 0.7184865864098854, 0.03374998176859239, 0.45617290909669045, 0.0, 0.6773097267519939, 0.0, 0.5679207496366352, 0.6115960602421397, 0.04872238312531689, 0.0717

In [19]:
i = [mean(dice_otsu_median_img_NIH3T3[0:17]), mean(dice_otsu_median_img_NIH3T3[18:35]), mean(dice_otsu_median_img_NIH3T3[36:53]), mean(dice_otsu_median_img_NIH3T3[54:71]), mean(dice_otsu_median_img_NIH3T3[72:89]), mean(dice_otsu_median_img_NIH3T3[90
:107])]
print(i)
max(i)

# median
# von filter [3, 4, 5, 10, 15, 20]  erhält: [0.502645808876782, 0.503612416524091, 0.5020557928423299, 0.5066893266757372, 0.50714522407093, 0.5041444827680592]
# 15 beste
#neuer test mit [12, 13, 14, 15, 16, 17] erhält:[0.5079665952706486, 0.5026104659405192, 0.5041593166592376, 0.50714522407093, 0.5084013249976619, 0.4955125467627179]

[0.5079665952706486, 0.5026104659405192, 0.5041593166592376, 0.50714522407093, 0.5084013249976619, 0.4955125467627179]


0.5084013249976619

### optimal filtersize median filter NIH3T3: 16

In [26]:
# Dice score - NIH3T3-dataset
## gaussian filter
dice_otsu_gauss_img_NIH3T3 = []

for i in range(0, len(otsu_gauss_img_NIH3T3)):
  dice_otsu_gauss_img_NIH3T3.append(dsc.dice(otsu_gauss_img_NIH3T3[i] , gt6_NIH3T3[i]))

print(dice_otsu_gauss_img_NIH3T3)

[0.9083236752442116, 0.8927423591404707, 0.811964308854292, 0.7195225589166834, 0.7488081630916394, 0.6541571607300559, 0.6354364461943924, 0.7136977554550509, 0.03176719971115433, 0.4571668251537708, 0.0, 0.6663548050593298, 0.00036662491216278147, 0.5700934301090961, 0.5992146063548041, 0.04682447917027272, 0.07881559982337308, 0.7663510722209179, 0.907993566318502, 0.8914642943015147, 0.8069728047725139, 0.7264822742303453, 0.743519705020811, 0.6420592182889145, 0.6207132472419397, 0.7030006087153718, 0.030260801481780123, 0.45019557165768614, 0.5434791870819309, 0.6601628541706102, 0.00044470409738147764, 0.5661672581481676, 0.5944399257544475, 0.04954240979132784, 0.07923756299585849, 0.7585414842260363, 0.9132327743602707, 0.8930596899436509, 0.8002268533922144, 0.712141561552414, 0.7381219160285925, 0.6360231055232585, 0.6234096772163845, 0.7026687257691597, 0.0311476030559253, 0.4424763383186015, 0.5405295941143352, 0.6536730382745934, 0.0005487422479269737, 0.5594109014644562,

In [27]:
i = [mean(dice_otsu_gauss_img_NIH3T3[0:17]), mean(dice_otsu_gauss_img_NIH3T3[18:35]), mean(dice_otsu_gauss_img_NIH3T3[36:53]), mean(dice_otsu_gauss_img_NIH3T3[54:71]), mean(dice_otsu_gauss_img_NIH3T3[72:89]), mean(dice_otsu_gauss_img_NIH3T3[90
:107])]
print(i)
max(i)


[0.502073882230633, 0.5303609408275944, 0.5274210806084128, 0.5234708498775114, 0.5207491425875806, 0.5185446137158545]


0.5303609408275944

In [ ]:
# filtersizes: 
#von filter [3, 4, 5, 10, 15, 20]  erhält:  [0.509766753006406, 0.5092587332586234, 0.5070860805568259, 0.5234708498775114, 0.5060017368659782, 0.48130440672068814]
# 10 bestes ergebnis
# neue filter: [7, 8 ,9 , 10, 11, 12] erhält: [0.502073882230633, 0.5303609408275944, 0.5274210806084128, 0.5234708498775114, 0.5207491425875806, 0.5185446137158545]
# 8 bestes ergebnis

### optimal filtersize gaussian NIH3T3: 8 

## N2DH-GOWT1

In [ ]:
# Load raw images from N2DH-GOWT1-dataset

img_N2DH_GOWT1 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)), r"data/Otsu_data/N2DH-GOWT1/img")
for f in os.listdir(path):
    img_N2DH_GOWT1.append(imread(os.path.join(path , f)))


# Load ground truth images from N2DH-GOWT1-dataset

gt_N2DH_GOWT1 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)), r"data/Otsu_data/N2DH-GOWT1/gt")
for f in os.listdir(path):
    gt_N2DH_GOWT1.append(imread(os.path.join(path , f)))

In [ ]:
# binarize

binary_gt_N2DH_GOWT1 = []
for i in range(0,len(gt_N2DH_GOWT1)):
    binary_gt_N2DH_GOWT1.append(binarize(gt_N2DH_GOWT1[i])) 

In [ ]:
#copy
copy_img_N2DH_GOWT1 = img_N2DH_GOWT1.copy()

In [ ]:
gt6_N2DH_GOWT1 = []

path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)), r"data/Otsu_data/N2DH-GOWT1/img")
y=0
while y<6:
    for f in os.listdir(path):
        gt6_N2DH_GOWT1.append(imread(os.path.join(path , f)))
    y+=1

print(shape(gt6_N2DH_GOWT1))

In [ ]:
# gaussian filter 

filter = [3, 4, 5, 10, 15, 20]
gauss_img_N2DH_GOWT1 = []

for j in range (0, len(filter)):

    for i in range(0,len(copy_img_N2DH_GOWT1)):
        gauss_img_N2DH_GOWT1.append(scipy.ndimage.gaussian_filter(copy_img_N2DH_GOWT1[i] , filter[j] ))


In [ ]:
# median filter

filter= [3, 4, 5, 10, 15, 20]
median_img_N2DH_GOWT1 = []

for j in range (0, len(filter)):

    for i in range(0,len(copy_img_N2DH_GOWT1)):
        gauss_img_N2DH_GOWT1.append(scipy.ndimage.gaussian_filter(copy_img_N2DH_GOWT1[i] , filter[j] ))

In [ ]:
# otsu
otsu_gauss_img_N2DH_GOWT1 = []
for i in range(0,len(gauss_img_N2DH_GOWT1)):
    otsu_gauss_img_N2DH_GOWT1.append(ot.otsu_thresholding(gauss_img_N2DH_GOWT1 [i] , 256))


In [ ]:
#dice

# Dice score - N2DH-GOWT1-dataset
## gaussian filter

dice_otsu_gauss_img_N2DH_GOWT1 = []
for j in range(0,len(otsu_gauss_img_N2DH_GOWT1)):
    dice_otsu_gauss_img_N2DH_GOWT1.append(dsc.dice(otsu_gauss_img_N2DH_GOWT1[j] , binary_gt_N2DH_GOWT1[j]))

print(dice_otsu_gauss_img_N2DH_GOWT1)



## N2DL-HeLa

In [ ]:
# Load raw images from N2DL-HeLa-dataset

img_N2DL_HeLa = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)), r"data/Otsu_data/N2DL-HeLa/img")
for f in os.listdir(path):
    img_N2DL_HeLa.append(imread(os.path.join(path , f)))


# Load ground truth images from N2Dl-HeLa-dataset

gt_N2DL_HeLa = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/N2DL-HeLa/gt")
for f in os.listdir(path):
    gt_N2DL_HeLa.append(imread(os.path.join(path , f)))

In [ ]:
binary_gt_N2DL_HeLa = []
for i in range(0,len(gt_N2DL_HeLa)):
    binary_gt_N2DL_HeLa.append(binarize(gt_N2DL_HeLa[i])) 

In [ ]:
copy_img_N2DL_HeLa = img_N2DL_HeLa.copy()

### median

In [ ]:
filter_g = [3, 4, 5, 10, 15, 20]
median_img_N2DL_HeLa = []

for j in range (0, len(filter_g[j])):

    for i in range(0,len(copy_img_N2DL_HeLa)):
        median_img_N2DL_HeLa.append(scipy.ndimage.median_filter(copy_img_N2DL_HeLa[i] , filter_g[j]))




TypeError: object of type 'int' has no len()

## otsu

In [ ]:
otsu_median_img_N2DL_HeLa = []
for i in range(0,len(median_img_N2DL_HeLa)):
    otsu_median_img_N2DL_HeLa.append(ot.otsu_thresholding(median_img_N2DL_HeLa [i] , 256))